In [1]:
from assets.models import Road

In [2]:
Road

assets.models.Road

In [3]:
from django.db.models import Func

In [5]:
from csv_data_sources.models import CsvData

In [2]:
CsvData.objects.first()

django.db.backends:DEBUG (0.001) SELECT "csv_data_sources_csvdata"."id", "csv_data_sources_csvdata"."source_id", "csv_data_sources_csvdata"."row_index", "csv_data_sources_csvdata"."data" FROM "csv_data_sources_csvdata" ORDER BY "csv_data_sources_csvdata"."id" ASC  LIMIT 1; args=()


In [4]:
Survey.objects.filter(values__has_key='roughness').count()

0

In [45]:
# Annotate the Survey objects with the "csv_data_row_index" property of JSON

annotate_row_ids = Survey.objects.filter(values__has_key='roughness').annotate(
    csv_data_source=Func(F('values'), template='''( %(expressions)s -> 'csv_data_source_id')::integer '''),
    csv_row=Func(F('values'), template='''( %(expressions)s -> 'csv_data_row_index')::integer '''),
    previous_row = Func(F('values'), template='''( %(expressions)s -> 'csv_data_row_index')::integer - 1 '''),
    next_row = Func(F('values'), template='''( %(expressions)s -> 'csv_data_row_index')::integer + 1 '''),
)

# Subquery for the last point's chainage
sq = Subquery(
    annotate_row_ids.filter(
        csv_data_source = OuterRef('csv_data_source'),
        csv_row=OuterRef('previous_row')
    ).values('chainage_end')
)

annotate_row_ids.annotate(previous_chainage_end = sq).values('chainage_start', 'chainage_end', 'previous_chainage_end')


django.db.backends:DEBUG (0.024) SELECT "assets_survey"."chainage_start", "assets_survey"."chainage_end", (SELECT U0."chainage_end" FROM "assets_survey" U0 WHERE (U0."values" ? 'roughness' AND ( U0."values" -> 'csv_data_source_id')::integer  = (( "assets_survey"."values" -> 'csv_data_source_id')::integer ) AND ( U0."values" -> 'csv_data_row_index')::integer  = (( "assets_survey"."values" -> 'csv_data_row_index')::integer - 1 ))) AS "previous_chainage_end" FROM "assets_survey" WHERE "assets_survey"."values" ? 'roughness'  LIMIT 21; args=('roughness', 'roughness')


<SurveyQuerySet [{'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage_end': Decimal('30316.97893'), 'previous_chainage_end': Decimal('30316.97893')}, {'chainage_start': Decimal('30316.97893'), 'chainage

In [1]:
import os
from django.core import management
from pathlib import Path
paths = []

gitdir = ('/home/josh/github/catalpainternational/estrada-data-sources')

for source_dir in [
    Path(gitdir) / 'csv' / 'Roughness-Survey-2019-National-Roads',
    Path(gitdir) / 'csv' / 'Roughness-Survey-2019-Municipal-Roads'
]:
    for  file_name in os.listdir(source_dir):
        paths.append(source_dir / file_name)

management.call_command('import_csv_source', 'roughness', *paths)


Deleted old roughness csv/Roughness-Survey-2019-National-Roads/A02.csv objects (691, {'csv_data_sources.CsvData': 690, 'csv_data_sources.CsvDataSource': 1})
Stored roughness csv source csv/Roughness-Survey-2019-National-Roads/A02.csv
Added roughness 690 csv source rows
Deleted old roughness csv/Roughness-Survey-2019-National-Roads/A01.csv objects (2142, {'csv_data_sources.CsvData': 2141, 'csv_data_sources.CsvDataSource': 1})
Stored roughness csv source csv/Roughness-Survey-2019-National-Roads/A01.csv
Added roughness 2141 csv source rows
Deleted old roughness csv/Roughness-Survey-2019-National-Roads/A13.csv objects (276, {'csv_data_sources.CsvData': 275, 'csv_data_sources.CsvDataSource': 1})
Stored roughness csv source csv/Roughness-Survey-2019-National-Roads/A13.csv
Added roughness 275 csv source rows
Deleted old roughness csv/Roughness-Survey-2019-National-Roads/A04.csv objects (484, {'csv_data_sources.CsvData': 483, 'csv_data_sources.CsvDataSource': 1})
Stored roughness csv source cs

In [1]:
%%time
RoughnessSurvey.objects.clear_surveys()
RoughnessSurvey.objects.make_surveys()

CPU times: user 2.59 s, sys: 44 ms, total: 2.63 s
Wall time: 40.2 s


In [2]:
Survey.objects.filter(values__has_key='source_roughness').count()

12379

In [3]:
Survey.objects.filter(chainage_start__gt=F('chainage_end'), values__has_key='source_roughness').values('chainage_start', 'chainage_end')

<SurveyQuerySet []>

In [4]:
Survey.objects.filter(chainage_start__gte=F('chainage_end')).count()

2152

In [5]:
Survey.objects.filter(values__has_key='roughness').count()

0

In [6]:
%%time
from assets.managers import update_roughness_survey_values
update_roughness_survey_values()

CPU times: user 8.82 ms, sys: 23 µs, total: 8.84 ms
Wall time: 544 ms


12379

In [7]:
%%time
from assets.managers import update_roughness_chainage_values
update_roughness_chainage_values()

KeyboardInterrupt: 

In [14]:
from django.db.models import Func

In [101]:
from django.db import models
from django.db.models import Func
from django.db.models.functions import Cast

class DataSourceId(Func):
    template = "(values -> 'csv_data_source_id'):: integer"
    default_alias = "csv_data_source"
    output_field = models.IntegerField()
    
class RowIndexId(Func):
    template = "(values -> 'csv_data_row_index'):: integer"
    default_alias = "csv_data_row"
    output_field = models.IntegerField()
    
# anno = Survey.objects.filter(values__has_key='roughness').annotate(innervalues=F('values')).annotate(DataSourceId(), RowIndexId()).annotate(next_csv_data_row=F('csv_data_row') + 1)


anno = Survey.objects.filter(values__has_key='roughness').annotate(
    DataSourceId(),
    RowIndexId(),
    prev_row = RowIndexId() - 1
).values(
    DataSourceId.default_alias,
    RowIndexId.default_alias,
    'prev_row',
    'chainage_start'
)

anno.annotate(chainage = Subquery(anno.filter(
    **{
        'prev_row': F('%s' % (RowIndexId.default_alias,)),
        RowIndexId.default_alias: F('%s' % (DataSourceId.default_alias,)),
    }
).values('chainage_start'))).values(
    'id',
    'chainage',
    'chainage_start'
).query.sql_with_params()


# print(anno)
#.annotate(next_inner_rowsource_id=F('inner_csv_data_row') + 1).annotate(
#    DataSourceId(),
#    RowIndexId()
#)
# anno.values('dsic', 'ridc')
# Survey.objects.annotate(DataSourceId(), RowIndexId()).annotate(next_row_start_chainage = sq_next_start).values('chainage_start', 'id', 'next_row_start_chainage').query.sql_with_params()

('SELECT "assets_survey"."id", "assets_survey"."chainage_start", (SELECT U0."chainage_start" FROM "assets_survey" U0 WHERE (U0."values" ? %s AND (values -> \'csv_data_row_index\'):: integer = ((values -> \'csv_data_source_id\'):: integer) AND ((values -> \'csv_data_row_index\'):: integer - %s) = ((values -> \'csv_data_row_index\'):: integer))) AS "chainage" FROM "assets_survey" WHERE "assets_survey"."values" ? %s',
 ('roughness', 1, 'roughness'))